In [2]:
from math import sqrt, isnan
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.data import Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from util import metrics
from util.load_data import load_data
from util.evaluation import compute_tradeoff

from madras_laftr.models import UnfairMLP

In [3]:
batch_size = 64
epochs = 100
lr = 0.001
opt = Adam(learning_rate=lr)

In [4]:
header = "model_name", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

test_loop = 1

## Load Data

In [5]:
x_train, y_train, a_train = load_data('adult', 'train')
raw_data = (x_train, y_train, a_train)

In [6]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [7]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 113), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [8]:
x_valid, y_valid, a_valid = load_data('adult', 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [9]:
x_test, y_test, a_test = load_data('adult', 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Train Lopp

In [10]:
def train(unfair_clas, X, Y, optimizer):
    
    with tf.GradientTape() as tape0:
        
        unfair_clas(X, Y, training=True) #to compute the foward
        current_loss = unfair_clas.loss #current loss
    
    grads = tape0.gradient(current_loss, unfair_clas.variables)
    
    optimizer.apply_gradients(zip(grads, unfair_clas.variables))

In [11]:
def train_loop(unfair_clas, train_dataset, epochs, optmizer):
    
    print("> Epoch | Class Loss | Class Acc")

    x_train, y_train, a_train = raw_data
    l = y_train.shape[0]
    l = (l//batch_size) * batch_size

    for epoch in range(epochs):
        Y_hat = None
        X_hat = None
        batch_count = 1
        
        for X, Y, A in train_dataset:
            
            train(model, X, Y, optmizer)

            if batch_count == 1:
                Y_hat = model.Y_hat
                batch_count += 1
                
            else:
                Y_hat = tf.concat([Y_hat, model.Y_hat], 0)

        clas_loss = tf.reduce_mean(model.loss)
        clas_acc = metrics.accuracy(y_train[:l], tf.math.round(Y_hat))

        print("> {} | {} | {}".format(
            epoch+1,
            clas_loss,
            clas_acc))

## Evaluation

In [12]:

def evaluation(model, valid_data):
    Y_hat = None
    batch_count = 1
    
    for X, Y, A in valid_data:
        
        model(X, Y, A)
        
        if batch_count == 1:
            Y_hat = model.Y_hat
            batch_count += 1
        else:
            Y_hat = tf.concat([Y_hat, model.Y_hat], 0)
    
    return Y_hat

In [13]:
def compute_metrics(Y, Y_hat, A):
    Y_hat = tf.math.round(Y_hat)
    
    clas_acc = metrics.accuracy(Y, Y_hat)

    print("> Class Acc")
    print("> {}".format(clas_acc))

    dp = metrics.DP(Y_hat.numpy(), A)
    deqodds = metrics.DEqOdds(Y, Y_hat.numpy(), A)
    deqopp = metrics.DEqOpp(Y, Y_hat.numpy(), A)

    print("> DP | DI | DEOPP")
    print("> {} | {} | {}".format(dp, deqodds, deqopp))

    tp = metrics.TP(Y, Y_hat.numpy())
    tn = metrics.TN(Y, Y_hat.numpy())
    fp = metrics.FP(Y, Y_hat.numpy())
    fn = metrics.FN(Y, Y_hat.numpy())

    print('> Confusion Matrix \n' +
                'TN: {} | FP: {} \n'.format(tn, fp) +
                'FN: {} | TP: {}'.format(fn, tp))

    confusion_matrix = np.array([[tn, fp],
                                [fn, tp]])

    m = [metrics.TN, metrics.FP, metrics.FN, metrics.TP]
    metrics_a0 = [0, 0, 0, 0]
    metrics_a1 = [0, 0, 0, 0]
    for i in range(len(m)):
        metrics_a0[i] = metrics.subgroup(m[i], A, Y, Y_hat.numpy())
        metrics_a1[i] = metrics.subgroup(m[i], 1 - A, Y, Y_hat.numpy())

    print('> Confusion Matrix for A = 0 \n' +
            'TN: {} | FP: {} \n'.format(metrics_a0[0], metrics_a0[1]) +
            'FN: {} | TP: {}'.format(metrics_a0[2], metrics_a0[3]))

    print('> Confusion Matrix for A = 1 \n' +
            'TN: {} | FP: {} \n'.format(metrics_a1[0], metrics_a1[1]) +
            'FN: {} | TP: {}'.format(metrics_a1[2], metrics_a1[3]))

    confusion_matrix = np.array([[tn, fp],
                                [fn, tp]])

    return clas_acc, confusion_matrix, dp, deqodds, deqopp, metrics_a0, metrics_a1

## Testing

In [14]:
for i in range(test_loop):
    opt = Adam(learning_rate=lr)

    model = UnfairMLP(xdim, zdim, ydim)
    ret = train_loop(model, train_data, epochs, opt)
    Y_hat = evaluation(model, valid_data)
    
    clas_acc, confusion_matrix, dp, deqodds, deqopp, metrics_a0, metrics_a1  = compute_metrics(y_valid, Y_hat, a_valid)
    
    fair_metrics = (dp, deqodds, deqopp)
    
    tradeoff = []
    
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))
    
    result = ['UnfairNN', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Class Acc
> 1 | 0.3769717216491699 | 0.7152271220159151
> 2 | 0.4737511873245239 | 0.8059515915119364
> 3 | 0.39866459369659424 | 0.8124171087533156
> 4 | 0.3726392984390259 | 0.8185510610079576
> 5 | 0.31772178411483765 | 0.8277934350132626
> 6 | 0.3403315246105194 | 0.8387350795755968
> 7 | 0.3272055387496948 | 0.8358338859416445
> 8 | 0.34666118025779724 | 0.831191976127321
> 9 | 0.34127357602119446 | 0.8358338859416445
> 10 | 0.3081939220428467 | 0.8373259283819628
> 11 | 0.29426077008247375 | 0.8398126657824934
> 12 | 0.36504411697387695 | 0.8297413793103449
> 13 | 0.3192847967147827 | 0.8294098143236074
> 14 | 0.3301752209663391 | 0.8342175066312998
> 15 | 0.29625654220581055 | 0.8391909814323607
> 16 | 0.2960474193096161 | 0.8402685676392573
> 17 | 0.3200136423110962 | 0.8341346153846154
> 18 | 0.2954990863800049 | 0.8359996684350133
> 19 | 0.30666470527648926 | 0.837616047745358
> 20 | 0.30616408586502075 | 0.837823275862069
> 21 | 0.306904137134552 | 0.8

## Saving into DF then CSV

In [15]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,UnfairNN,0.8376,0.782787,0.910585,0.930984,0.809267,0.872569,0.881826,1577.0,90.0,72.0,140.0,2384.0,467.0,348.0,938.0


In [16]:
#result_df.to_csv('results/teste_unfair_nn-100.csv')